In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho) #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto)#el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 1:alto, x in 1:ancho]
end

function moneda(N)
    ceil(Int64, round(rand(Uniform(-N-0.499, N+0.499))))
end

function paleta(mi_foto, paso, tamaño)
    aesthetics = load(mi_foto)
    alto, ancho =size(aesthetics)
    x0 = ceil(Int64, rand(Uniform(0,ancho)))
    #x0 = ceil(Int64, ancho/3)
    y0 = ceil(Int64, rand(Uniform(0,alto)))
    #y0 = ceil(Int64, alto/2)
    paleta = zeros(RGB{N0f8}, tamaño)
    for i = 1:length(paleta)
        paleta[i] = aesthetics[mod(y0,alto)+1,mod(x0,ancho)+1]
        x0 = x0 + moneda(paso)
        y0 = y0 + moneda(paso)
    end
    return paleta
end

# Conjuntos de Julia cuadráticos

En este ejemplo veremos como pintar el *interior* del Conjunto de Julia relleno, es decir, vamos a usar el algoritmo de trampas para pintar los componentes acotados del conjunto de Fatou para polinomios cuadráticos en la familia $p_{\lambda}(z) = z^2 + \lambda z$.

La información que usaremos sobre esta familia es que el $0$ es un punto fijo con multiplicador $\lambda$.

Para estos dibujos usaremos la paleta generada por una caminata aleatoria en una imagen.

In [ ]:
mi_paleta = paleta("aesthetic.jpg", 5, 1000)

Vamos a usar una trampa en el punto fijo que conocemos, el cero, y en principio solo vamos a considerar parametros $|\lambda| < 1$. La región que usaremos para la trampa es un disco de radio ``eps`` centrado en $0$.

In [ ]:
function trampa_atractor(Z, λ, colores, eps=1.0e-5, TOPE=1000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps
            n = 1 + mod(cont, length(colores))
            return colores[n]
        elseif abs(Z) > 4.0
            return RGB{Float32}(1.0, 1.0, 1.0) #color blanco
        end
        Z = Z*(Z + λ)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro
end

function julia2_atractor(λ, colores, eps=1.0e-5, TOPE=1000, radio=2.0, centro=0.0, alto=1000, ancho=1000)
    broadcast(Z -> trampa_atractor(Z, λ, colores, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
julia2_atractor(0.25+0.25im, mi_paleta)

Consideremos ahora el caso $|\lambda| = 1$, es decir, $\lambda = e^{2 \pi i \theta}$, donde $\theta \in [0,1]$.

Comenzaremos con el caso parábolico, $\theta \in \mathbb{Q}$. La trampa que usaremos será un sector contenido en un pétalo atractor: centrado en 0, de radio ``eps``, en el angulo ``dir`` y de ancho ``wdt``. 

In [ ]:
function trampa_parabolico(Z, λ, dir, wdt, colores, eps=0.05, TOPE=5000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps && (angle(Z) > dir - wdt && angle(Z) < dir + wdt)
            n = 1 + mod(cont, length(colores))
            return colores[n]
        elseif abs(Z) > 4.0
            return RGB{Float32}(0.0, 0.0, 0.0) #color negro
        end
        Z = Z*(Z + λ)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro
end

Para estimar la dirección y el tamaño del sector que se usara como trampa podemos *perseguir* el punto critico, $-\lambda /2$, iterando varias veces (por ejemplo, tantas veces como indique ``TOPE``) el punto crítico obtendremos un punto cerca del punto fijo. Usamos el argumento de esta imagen del punto critico como direccion para el sector y usamos un ancho proporcional a ``eps`` y al numero de petalos. 

La función ``julia2_parabolico`` toma como parametros el numerador $p$ y el denominador $q$ del multiplicador $\lambda = e^{2 \pi i p/q}$

In [ ]:
function julia2_parabolico(p, q, colores, eps=0.05, TOPE=5000, radio=2.0, centro=0.0, alto=1000, ancho=1000)
    λ = cos(2.0*pi*p/q) + sin(2.0*pi*p/q)*1.0im
    cr = -0.5*λ
    for i = 1:TOPE
        if abs(cr) < eps
            break
        end
        cr = cr*(cr + λ)
    end
    dir = angle(cr)
    wdt = eps/q
    broadcast(Z -> trampa_parabolico(Z, λ, dir, wdt, colores, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
julia2_parabolico(2,3, mi_paleta, 0.4, 500, 1.35, 0.2+0.4im)

In [ ]:
julia2_parabolico(5,7, mi_paleta, 0.2, 5000, 0.5)

Ahora consideremos el caso $\lambda = e^{2 \pi i \theta}$ con $\theta$ irracional (de Brjuno) de manera que haya un disco de Siegel. La región que usaremos para la trampa será un disco centrado en el punto fijo 0 de radio ``rad``. Este radio deberá ser suficentemente grande para que *atrape* todas las oribitas que llegan al dominio de rotación, pero no tan grande como para que contenga puntos del conjunto de Julia. 

En el caso de la familia cuadrática, sabemos que el punto critico se acumula en la frontera del disco de Siegel. Usando esta información, podemos estimar un buen ``rad`` para la trampa como el mínimo de las distancias de (un pedazo) la órbita del punto critico al cero. 

In [ ]:
function trampa_siegel(Z, λ, rad, colores, TOPE=5000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < rad
            n = 1 + mod(cont, length(colores))
            return colores[n]
        elseif abs(Z) > 4.0
            return RGB{Float32}(0.0, 0.0, 0.0) #color negro si diverge
        end
        Z = Z*(Z + λ)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro si no concluye
end

function julia2_siegel(θ, colores, TOPE=5000, radio=2.0, centro=0.0, alto=1000, ancho=1000)
    λ = cos(2.0*pi*θ) + sin(2.0*pi*θ)*1.0im
    cr = -0.5*λ
    rad = 1000.0
    for i = 1:TOPE
        cr = cr*(cr + λ)
        rad > abs(cr) ? rad = abs(cr) : nothing
    end
    broadcast(Z -> trampa_siegel(Z, λ, rad, colores, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
julia2_siegel(0.5*(sqrt(5.0) - 1.0), mi_paleta, 1000, 1.0)